<a href="https://colab.research.google.com/github/RezuanaHaque/Keras-Tuner-Hyperparameter-Tuning-How-To-Select-Hidden-Layers-And-Number-of-Hidden-Neurons-In-ANN/blob/main/Keras_Tuner_Hyperparameter_Tuning_How_To_Select_Hidden_Layers_And_Number_of_Hidden_Neurons_In_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [3]:
url = "https://raw.githubusercontent.com/RezuanaHaque/Keras-Tuner-Hyperparameter-Tuning-How-To-Select-Hidden-Layers-And-Number-of-Hidden-Neurons-In-ANN/main/Real_Combine.csv"
data = pd.read_csv(url)
data.head(5)

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
data.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [5]:
data.dropna()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [6]:
data.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [7]:
data["PM 2.5"].isnull().values.any()

True

In [8]:
data = data.dropna()

In [9]:
data["PM 2.5"].isnull().values.any()

False

In [10]:
x = data.iloc[: , :-1]
y = data.iloc[: , -1]

### Hyperparameters


*  How many number of hidden layers we should have?
*  How many number of neurons we should have in hidden layers?
*  Learning Rate





In [55]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
        if hp.Boolean("dropout"):
          model.add(layers.Dropout(rate=0.5))
    # Tune whether to use dropout.

    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-5])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [56]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory='projects',
    project_name='Air Quality Index')

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)


In [58]:

tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 06s]
val_mean_absolute_error: 91.24145126342773

Best val_mean_absolute_error So Far: 47.61000442504883
Total elapsed time: 00h 00m 23s
INFO:tensorflow:Oracle triggered exit


In [59]:

tuner.results_summary()

Results summary
Results in projects/Air Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 352
dropout: False
units_1: 288
learning_rate: 0.01
units_2: 192
units_3: 160
units_4: 64
Score: 47.61000442504883
Trial summary
Hyperparameters:
num_layers: 4
units_0: 320
dropout: False
units_1: 288
learning_rate: 0.01
units_2: 160
units_3: 160
units_4: 128
Score: 48.09172248840332
Trial summary
Hyperparameters:
num_layers: 5
units_0: 512
dropout: False
units_1: 480
learning_rate: 0.001
units_2: 64
units_3: 512
units_4: 416
Score: 57.732666015625
Trial summary
Hyperparameters:
num_layers: 5
units_0: 64
dropout: True
units_1: 384
learning_rate: 1e-05
units_2: 32
units_3: 32
units_4: 32
Score: 85.65097045898438
Trial summary
Hyperparameters:
num_layers: 9
units_0: 64
dropout: True
units_1: 192
learning_rate: 0.01
units_2: 256
units_3: 320
units_4: 384
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: 91.24145126342773
